In [1]:
from sklearn.datasets import load_wine
from datetime import timedelta
import pandas as pd
import numpy as np

In [2]:
read_data_file = pd.read_pickle('../hive.pkl')
df = pd.DataFrame(read_data_file)

In [3]:
#X = df[['commit_befor ', 'dispen_bf', 'object_bf', 'bloater_bf', 'open_time',]]
X = df[['commit_befor ', 'commit_after', 'cal_smell', 'dispen_bf', 'object_bf',
       'bloater_bf', 'dispen_at', 'object_at', 'bloater_af', 'open_time',
       'closed_time','changed_file', 'additions', 'deletions',
       'dev']]
y = df['cal_time_binaly']

In [4]:
X_train ,X_test ,y_train , y_test = X[:20], X[20:],y[:20],y[20:]

In [5]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))

In [6]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf , X , y , cv=5 , scoring= "accuracy")

array([1.        , 0.33333333, 0.83333333, 0.5       , 0.5       ])

In [7]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(clf,X,y,cv=5)
y_pred

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 1], dtype=int64)

In [8]:
y_pred_test =cross_val_predict(clf,X_test,y_test,cv=5)
y_pred_test

C:\Users\Beauz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [9]:
confusion_matrix(y,y_pred)

array([[ 7,  7],
       [ 4, 12]], dtype=int64)

In [10]:
from sklearn.metrics import precision_score, recall_score, f1_score
print('precision = ', precision_score(y,y_pred))
print('recall_score = ', recall_score(y,y_pred))
print('f1_score',f1_score(y,y_pred))

precision =  0.631578947368421
recall_score =  0.75
f1_score 0.6857142857142857


In [11]:
frame={"y":y,"y_pred": y_pred}
pd.DataFrame(frame).sum()

y         16
y_pred    19
dtype: int64

In [12]:
from sklearn.model_selection import GridSearchCV

parameters = {
  
    'C': [0.01 , 0.1 ,1],
    'gamma' : ['scale' , 'auto'],
    'random_state':[2]
  
}
svc = SVC()
clf = GridSearchCV(svc , parameters , cv=3 , scoring='f1_macro')
clf.fit(X_test,y_test)

GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': [0.01, 0.1, 1], 'gamma': ['scale', 'auto'],
                         'random_state': [2]},
             scoring='f1_macro')

In [13]:
y_pred_clf = clf.best_estimator_.predict(X)
y_pred_clf

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [14]:
from sklearn.metrics import precision_score, recall_score, f1_score
print('precision = ', precision_score(y,y_pred_clf ,average='macro'))
print('recall_score = ', recall_score(y,y_pred_clf ,average='macro'))
print('f1_score',f1_score(y,y_pred_clf ,average='macro'))

precision =  0.26666666666666666
recall_score =  0.5
f1_score 0.3478260869565218


C:\Users\Beauz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
from matplotlib import pyplot as plt
from sklearn import svm
import numpy as np

def f_importances(coef, names):
    
    names = np.arange(1)
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

features_names = X.columns
svm1 = svm.SVC(kernel='linear',C=1)
svm1.fit(X_test, y_test)

SVC(C=1, kernel='linear')

In [16]:
svm1.coef_

array([[-8.77855187e-02, -2.75163333e-02,  6.02691854e-02,
        -3.45297481e-02,  0.00000000e+00, -5.18078189e-02,
        -1.44795171e-03,  7.26870788e-02, -8.14905864e-02,
        -5.30451794e-02, -2.14402840e-02,  7.66312400e-02,
         5.06830211e-03, -1.85293106e-02, -6.50521303e-17]])

In [17]:
import itertools
from itertools import product 

In [18]:
result = itertools.combinations(X ,2)

for item in result:
    df  = pd.DataFrame(item)
    X[list (item)]
    parameters = {
    'C': [0.01 , 0.1 ,1],
    'gamma' : ['scale' , 'auto'],
    'random_state':[2]
    }
    svc = SVC()
    clf = GridSearchCV(svc , parameters , cv=3 , scoring='f1_macro')
    clf.fit(X_train[list (item)],y_train)
    clf.predict(X_test[list (item)])
    #print('predict=',clf.predict(X_test[list (item)]))
    #print(X_test)
    #print("y_train = ",y_train)
    print(item)
    print("precision_score =",precision_score(clf.predict(X[list(item)]),y))
    print("recall_score =", recall_score(clf.predict(X[list(item)]),y))
    print("f_1=",f1_score(clf.predict(X[list(item)]),y,average='macro'))
    #print("f_1_X-test=",f1_score(clf.predict(X_test[list(item)]),y_test,average='macro'))
    

('commit_befor ', 'commit_after')
precision_score = 0.5625
recall_score = 1.0
f_1= 0.76
('commit_befor ', 'cal_smell')
precision_score = 0.9375
recall_score = 0.7142857142857143
f_1= 0.7532314923619272
('commit_befor ', 'dispen_bf')
precision_score = 0.25
recall_score = 0.8
f_1= 0.5238095238095237
('commit_befor ', 'object_bf')
precision_score = 0.0
recall_score = 0.0
f_1= 0.3181818181818182


C:\Users\Beauz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Beauz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('commit_befor ', 'bloater_bf')
precision_score = 0.0
recall_score = 0.0
f_1= 0.3181818181818182
('commit_befor ', 'dispen_at')
precision_score = 0.375
recall_score = 0.8571428571428571
f_1= 0.6122209165687424
('commit_befor ', 'object_at')
precision_score = 0.25
recall_score = 0.8
f_1= 0.5238095238095237
('commit_befor ', 'bloater_af')
precision_score = 0.9375
recall_score = 0.7142857142857143
f_1= 0.7532314923619272
('commit_befor ', 'open_time')
precision_score = 0.6875
recall_score = 1.0
f_1= 0.8316498316498315
('commit_befor ', 'closed_time')
precision_score = 0.5625
recall_score = 0.75
f_1= 0.6651785714285715
('commit_befor ', 'changed_file')
precision_score = 0.0625
recall_score = 1.0
f_1= 0.38440492476060195
('commit_befor ', 'additions')
precision_score = 0.8125
recall_score = 0.5909090909090909
f_1= 0.5693779904306221
('commit_befor ', 'deletions')
precision_score = 0.0
recall_score = 0.0
f_1= 0.3181818181818182
('commit_befor ', 'dev')
precision_score = 0.3125
recall_score =

C:\Users\Beauz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('commit_after', 'cal_smell')
precision_score = 0.6875
recall_score = 0.9166666666666666
f_1= 0.7991071428571429
('commit_after', 'dispen_bf')
precision_score = 0.5625
recall_score = 0.9
f_1= 0.7285067873303168
('commit_after', 'object_bf')
precision_score = 0.0
recall_score = 0.0
f_1= 0.3181818181818182
('commit_after', 'bloater_bf')
precision_score = 0.4375
recall_score = 1.0
f_1= 0.6827262044653349


...

C:\Users\Beauz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('closed_time', 'dev')
precision_score = 0.5625
recall_score = 0.6
f_1= 0.5661846496106784
('changed_file', 'additions')
precision_score = 0.75
recall_score = 0.6
f_1= 0.5833333333333333
('changed_file', 'deletions')
precision_score = 0.0
recall_score = 0.0
f_1= 0.3181818181818182
('changed_file', 'dev')
precision_score = 0.75
recall_score = 0.9230769230769231
f_1= 0.8331479421579533


C:\Users\Beauz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('additions', 'deletions')
precision_score = 0.5625
recall_score = 0.6428571428571429
f_1= 0.6000000000000001
('additions', 'dev')
precision_score = 0.8125
recall_score = 0.5909090909090909
f_1= 0.5693779904306221
('deletions', 'dev')
precision_score = 0.0
recall_score = 0.0
f_1= 0.3181818181818182


C:\Users\Beauz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
